#### School of Tomorrow

[Oregon Curriculum Network](http://4dsolutions.net/ocn/)<br/>
[School of Tomorrow](School_of_Tomorrow.ipynb)



# BUILDING A VOLUMES TABLE

### Using Koski Identities

In [1]:
import sys, os
sys.version

'3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]'

In [2]:
os.getcwd()

'/Users/kirbyurner/Documents/School_of_Tomorrow'

The School of Tomorrow landing page currently introduces a code cell like this:

In [3]:
"""
From David Koski's paper:
Revisiting R.B. Fuller's S & E Modules
--------
The concentric hierarchy can be described in terms of 
phi-scaled S modules: 

Tetrahedron:  21S +  5s3 =  5S3 + 1S = 1S6 + 1S3
Skew Icosa:   60S + 20s3 
Cube       :  72S + 15s3 = 15S3 + 3S = 3S6 + 3S3
Octahedron :  84S + 20s3 = 20S3 + 4S = 4S6 + 4S3 
Rh Triac   : 105S + 25s3 = 25S3 + 5S = 5S6 + 5S3 (volume = 5) 
Rh Dodec   : 126S + 30s3 = 30S3 + 6S = 6S6 + 6S3
Pent Dodeca: 348E + 84e3
Icosahedron: 100E3 + 20E
VE         : 20; 420S + 100s3
SuperRT    : 20 * Syn3; 480E + 120e3
--------

On-line:  http://coffeeshopsnet.blogspot.com/2017/06/koski-paper.html
"""

from math import sqrt as rt2

φ     = (rt2(5)+1)/2  # golden ratio
Syn3  = rt2(9/8)      # not to be confused with Smod

Smod  = (φ**-5) / 2   # home base Smod
smod3 = Smod * φ**-3  # small s, phi down
Smod3 = Smod * φ**3   # capital s, phi up
Smod6 = Smod3 * φ**3  # phi up yet again

Cubocta   = 20
SuperRT   = Syn3 * Cubocta

Emod3     = SuperRT / 120  # Emod phi up
Emod      = (rt2(2) / 8) * (φ**-3)  # home base Emod
emod3     = Emod * φ**-3  # Emod phi down

S_factor  = Smod / Emod  # 2*sqrt(7-3*sqrt(5))
T_factor  = pow(3/2, 1/3) * rt2(2) / φ
Tmod      = 1/24

FiveVEs   = 480 * Smod + 280 * smod3
Icosa     = 100 * Emod3 + 20 * Emod
P_Dodeca  = 348 * Emod  + 84 * emod3
FiveOctas = 132 * Smod  + 36 * smod3
RD        =   6 * Smod6 +  6 * Smod3 
RT5plus   = 120 * Emod  
RT5       =   5 * Smod6 +  5 * Smod3  
Octa      =   4 * Smod6 +  4 * Smod3
Cube      =   3 * Smod6 +  3 * Smod3
SmallVE   = Cubocta * 1/8  # half D edges (=R)
SkewIcosa =  60 * Smod  + 20 * smod3 
Tetra     =   5 * Smod3 + Smod

print("Shape           Volume      Check")
print(f"Five VEs      : {FiveVEs:10.6f}")
print(f"SuperRT       : {SuperRT:10.6f}")
print(f"Cubocta       : {Cubocta:10.6f}")
print(f"Icosa         : {Icosa:10.6f}  {20 * 1/S_factor:10.6f}")
print(f"P Dodeca      : {P_Dodeca:10.6f}")
print(f"Five Octas    : {FiveOctas:10.6f}")
print(f"Rh Dodeca (RD): {RD:10.6f}")
print(f"RT5+          : {RT5plus:10.6f}  {SuperRT * φ**-3:10.6f}")
print(f"RT5           : {RT5:10.6f}  {120 * Tmod:10.6f}")
print(f"Octa          : {Octa:10.6f}")
print(f"Cube          : {Cube:10.6f}")
print(f"Skew Icosa    : {SkewIcosa:10.6f}  {SmallVE * S_factor**2:10.6f}")
print(f"Small VE      : {SmallVE:10.6f}")
print(f"Tetra         : {Tetra:10.6f}")
print("-" * 20)
print(f"Emod3         : {Emod3:10.6f}")
print(f"Emod          : {Emod:10.6f}")
print(f"emod3         : {emod3:10.6f}")

Smod_check = (Octa - SkewIcosa) / 24
print(f"Tmod          : {Emod * (1 / T_factor**3):10.6f}")
print(f"Smod6         : {Smod6:10.6f}")
print(f"Smod3         : {Smod3:10.6f}")
print(f"Smod          : {Smod:10.6f}  {Smod_check:10.6f}")
print(f"smod3         : {smod3:10.6f}")

Shape           Volume      Check
Five VEs      :  24.620860
SuperRT       :  21.213203
Cubocta       :  20.000000
Icosa         :  18.512296   18.512296
P Dodeca      :  15.350018
Five Octas    :   6.334369
Rh Dodeca (RD):   6.000000
RT5+          :   5.007758    5.007758
RT5           :   5.000000    5.000000
Octa          :   4.000000
Cube          :   3.000000
Skew Icosa    :   2.917961    2.917961
Small VE      :   2.500000
Tetra         :   1.000000
--------------------
Emod3         :   0.176777
Emod          :   0.041731
emod3         :   0.009851
Tmod          :   0.041667
Smod6         :   0.809017
Smod3         :   0.190983
Smod          :   0.045085    0.045085
smod3         :   0.010643


We might want to preview / advertise more of the Python ecosystem's capabilities by making a corresponding pandas DataFrame and populating it not only with floating point expressions of relative volume, but algebraic expressions as well. For the latter, we turn to sympy.

In [4]:
import pandas as pd
import sympy as sy
from sympy import Rational, Integer

In [5]:
rt2 = sy.sqrt

In [6]:
φ     = (rt2(5)+1)/2            # golden ratio
Syn3  = rt2(Rational(9,8))      # not to be confused with Smod

In [7]:
from tetravolume import B, E, A, S, T
Bmod = B()
Amod = A()
Smod = S()
Kmod = T() * Rational(3,2) ** Rational(1,3) # multiply edges by 3rd root of 3/2
Emod = E()
Tmod = T()

In [8]:
Kmod.ivm_volume().simplify()

1/16

In [9]:
Bmod.__dict__

{'a': sqrt(2)/2,
 'a2': 1/2,
 'b': sqrt(6)/4,
 'b2': 3/8,
 'c': 1/2,
 'c2': 1/4,
 'd': sqrt(6)/12,
 'd2': 1/24,
 'e': sqrt(2)/4,
 'e2': 1/8,
 'f': 1/2,
 'f2': 1/4,
 'AB': sqrt(2)/2,
 'AC': sqrt(6)/4,
 'AD': 1/2,
 'BC': sqrt(6)/12,
 'CD': sqrt(2)/4,
 'BD': 1/2,
 'BAC': acos(5*sqrt(3)/9),
 'CAD': acos(sqrt(6)/3),
 'BAD': pi/4,
 'ABC': acos(sqrt(3)/3),
 'CBD': acos(sqrt(6)/3),
 'ABD': pi/4,
 'ACB': acos(-1/3),
 'ACD': acos(sqrt(3)/3),
 'BCD': acos(-sqrt(3)/3),
 'ADC': pi/2,
 'BDC': acos(2*sqrt(2)/3),
 'ADB': pi/2}

In [10]:
Amod.__dict__

{'a': sqrt(6)/12,
 'a2': 1/24,
 'b': sqrt(6)/4,
 'b2': 3/8,
 'c': sqrt(2)/4,
 'c2': 1/8,
 'd': sqrt(3)/3,
 'd2': 1/3,
 'e': 1/2,
 'e2': 1/4,
 'f': sqrt(3)/6,
 'f2': 1/12,
 'AB': sqrt(6)/12,
 'AC': sqrt(6)/4,
 'AD': sqrt(2)/4,
 'BC': sqrt(3)/3,
 'CD': 1/2,
 'BD': sqrt(3)/6,
 'BAC': acos(1/3),
 'CAD': acos(sqrt(3)/3),
 'BAD': acos(sqrt(3)/3),
 'ABC': pi/2,
 'CBD': pi/3,
 'ABD': pi/2,
 'ACB': acos(2*sqrt(2)/3),
 'ACD': acos(sqrt(6)/3),
 'BCD': pi/6,
 'ADC': pi/2,
 'BDC': pi/2,
 'ADB': acos(sqrt(6)/3)}

In [11]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'angles',
 'degrees',
 'dihedral',
 'dihedrals',
 'dump',
 'edges',
 'fig913_01',
 'fig986_421',
 'ivm_volume',
 'xyz_volume']

In [12]:
A().ivm_volume()

1/24

In [13]:
for lbl, v in zip(list("BASKET"), [Bmod,Amod,Smod,Kmod,Emod,Tmod]):
    print(lbl, ": ",  v.ivm_volume().simplify())

B :  1/24
A :  1/24
S :  -sqrt(5)*sqrt(2*sqrt(5) + 6) + 9*sqrt(2*sqrt(5) + 6)/4
K :  1/16
E :  -sqrt(2)/4 + sqrt(10)/8
T :  1/24


In [14]:
B1,A1,S1,K1,E1,T1 = Bmod.ivm_volume(), Amod.ivm_volume(), Smod.ivm_volume(), Kmod.ivm_volume(), Emod.ivm_volume(), Tmod.ivm_volume()

E3 = E1 * φ**3
S3 = S1 * φ**3
E6 = E3 * φ**3
S6 = S3 * φ**3
e3 = E1 * φ**-3
s3 = S1  * φ**-3

In [15]:
Super_RT  = Integer(20) * Syn3
VE        = Integer(20)
Icosa     = 100 * E3 + 20 * E1
P_Dodeca  = 348 * E1 + 84 * e3
RD        =   6 * (S6 + S3) 
RT5plus   = 120 * E1  
RT5       =   5 * (S6 + S3)  
Octa      =   4 * (S6 + S3) 
Cube      =   3 * (S6 + S3) 
SmallVE   =  VE * Rational(1,8)  # half D edges (=R)
SkewIcosa =  60 * S1 + 20 * s3 
Tetra     =   1 * (S6 + S3) 

In [16]:
from pandas import DataFrame

In [17]:
shapes = ['SuperRT', 'VE', 'Icosa', 'PD', 'RD', 
          'RT_E', 'RT_T', 'Octa', 'Cube', 'SmallVE',
          'SkewIcosa', 'Tetra', 'A', 'B', 'T', 'E', 'S', 'K']

volumes = [vol_expr.simplify() for vol_expr in 
          [Super_RT, VE, Icosa, P_Dodeca, RD, 
           RT5plus, RT5, Octa, Cube, SmallVE, 
           SkewIcosa, Tetra, A1, B1, T1, E1, S1, K1]]

In [18]:
volumes

[15*sqrt(2),
 20,
 5*sqrt(10)/2 + 15*sqrt(2)/2,
 3*sqrt(10)/2 + 15*sqrt(2)/2,
 -3*sqrt(2)*sqrt(sqrt(5) + 3)/2 + 3*sqrt(10)*sqrt(sqrt(5) + 3)/2,
 -30*sqrt(2) + 15*sqrt(10),
 -5*sqrt(2)*sqrt(sqrt(5) + 3)/4 + 5*sqrt(10)*sqrt(sqrt(5) + 3)/4,
 -sqrt(2)*sqrt(sqrt(5) + 3) + sqrt(10)*sqrt(sqrt(5) + 3),
 -3*sqrt(2)*sqrt(sqrt(5) + 3)/4 + 3*sqrt(10)*sqrt(sqrt(5) + 3)/4,
 5/2,
 -55*sqrt(2)*sqrt(sqrt(5) + 3) + 25*sqrt(10)*sqrt(sqrt(5) + 3),
 -sqrt(2)*sqrt(sqrt(5) + 3)/4 + sqrt(10)*sqrt(sqrt(5) + 3)/4,
 1/24,
 1/24,
 1/24,
 -sqrt(2)/4 + sqrt(10)/8,
 -sqrt(5)*sqrt(2*sqrt(5) + 6) + 9*sqrt(2*sqrt(5) + 6)/4,
 1/16]

In [19]:
newvols = [ ]
for v in volumes:
    r = v.evalf()
    if int(r) == float(r):
        newvols.append(Integer(r))
    else:
        newvols.append(v)
newvols    

[15*sqrt(2),
 20,
 5*sqrt(10)/2 + 15*sqrt(2)/2,
 3*sqrt(10)/2 + 15*sqrt(2)/2,
 6,
 -30*sqrt(2) + 15*sqrt(10),
 5,
 4,
 3,
 5/2,
 -55*sqrt(2)*sqrt(sqrt(5) + 3) + 25*sqrt(10)*sqrt(sqrt(5) + 3),
 1,
 1/24,
 1/24,
 1/24,
 -sqrt(2)/4 + sqrt(10)/8,
 -sqrt(5)*sqrt(2*sqrt(5) + 6) + 9*sqrt(2*sqrt(5) + 6)/4,
 1/16]

In [20]:
volumes_num = [vol.evalf() for vol in volumes]
volumes_num

[21.2132034355964,
 20.0000000000000,
 18.5122958682192,
 15.3500182080508,
 6.00000000000000,
 5.00775803133284,
 5.00000000000000,
 4.00000000000000,
 3.00000000000000,
 2.50000000000000,
 2.91796067500631,
 1.00000000000000,
 0.0416666666666667,
 0.0416666666666667,
 0.0416666666666667,
 0.0417313169277737,
 0.0450849718747371,
 0.0625000000000000]

In [21]:
volumes_table = DataFrame({'Volumes (num)': volumes_num,
                           'Volumes (alg)': newvols},
                            index = shapes)

In [22]:
pd.set_option('display.max_colwidth', None)

In [23]:
volumes_table

,Volumes (num),Volumes (alg)
SuperRT,21.2132034355964,15*sqrt(2)
VE,20.0000000000000,20
Icosa,18.5122958682192,5*sqrt(10)/2 + 15*sqrt(2)/2
PD,15.3500182080508,3*sqrt(10)/2 + 15*sqrt(2)/2
RD,6.00000000000000,6
RT_E,5.00775803133284,-30*sqrt(2) + 15*sqrt(10)
RT_T,5.00000000000000,5
Octa,4.00000000000000,4
Cube,3.00000000000000,3
SmallVE,2.50000000000000,5/2


In [24]:
Syn3

3*sqrt(2)/4